# TP-3 Diffusion Models

### Student information
Student: Balthazar Neveu

🔗 [Github](https://github.com/balthazarneveu/MVA24-delire) | [Online page for this class lab session](https://balthazarneveu.github.io/MVA24-delire)

⭐ [Online HTML version of this notebook](https://balthazarneveu.github.io/MVA24-delire/TP_3/tp_3.html)

# 


# DDPM: Denoising Diffusion Probabilistic Models

### Question 1
>  Use 
> - the definition of the forward process $x_t \sim \mathcal{N}( \sqrt{\bar{\alpha_t}}x_{0},\,(1-\bar{\alpha_t}) I)$ 
> - Tweedie's identity
>
>  to show that 
> $ \widehat{x_0}(x_t) = E[x_0|x_t] = \frac{1}{\sqrt{\bar\alpha_t}} \left( x_t + (1-\bar\alpha_t) \nabla \log p(x_t) \right) $.


__Tweedie's formula__
- $y = X+ N$
  - $y$ noisy version of clean image
  - $X$ with an additive white gaussian noise $N \sim \mathcal{N}(0, \sigma^2 I)$ (variance $\sigma^2$)
- MMSE denoiser $D^{*}_{\sigma^2}(y) = \mathbb{E}[X|y=X+N]$.
  - $D^{*}$ the star means this is the ideal maximum mean squared error denoiser.
  - This will be modeled using a denoiser neural network trained on the AWGN denoising problem using the MSE loss. 
  - We'll only get an approximation of $D^{*}$
- $\frac{1}{\sigma^2} (D^{*}_{\sigma^2} - I)(x) = \nabla \log p_{\sigma^2}(x)$  
  - where $p_{\sigma^2}(x)$ is the blurred version of the probability distribution of p by the Gaussian kernel of variance $\sigma^2$ .
  - $p_{\sigma^2} := p \star g_{\sigma^2}$
  - This means that the denoising residual is proportional to the score function (gradient of log propability) if the blurred distribution. The more noise level, the blurrier probability distribution.


__Result proof__
- From the forward process definition: 
  - (1) $x_t = \sqrt{\bar{\alpha_t}}x_{0} + N$ 
  - with $N \sim \mathcal{N}(0, (1-\bar{\alpha_t})I) $
  - Note: *$x_t$ is a noisy and "darker" (by a factor $\bar{\alpha_t}$) version of $x_{0}$. Making the image intensity tend to 0 is necessary to end up with a zero centered gaussian noise when $t \rightarrow \infty$*
- $ \widehat{x_0}(x_t) = E[x_0|x_t]$ 
  - $\widehat{x_0}(x_t)$ is the ideal MMSE denoiser result of $x_t$ .
- $ \widehat{x_0}(x_t) = E[x_0| \sqrt{\bar{\alpha_t}}x_{0} + N]$ by (1) the forward process definition

We first need  to adapt to be able to apply Tweedie's formula ($\bar{\alpha_t}x_{0}$  and $x_{0}$ appear..., let's make $X = \sqrt{\bar{\alpha_t}}X_{0}$ appear on both sides so we can apply the formula):

$$ \widehat{x_0}(x_t) = \frac{1}{\sqrt{\bar{\alpha_t}}} \mathbb{E}[\underbrace{\sqrt{\bar{\alpha_t}} x_0}_{X}| \underbrace{\sqrt{\bar{\alpha_t}}x_{0}}_{X} + N = x_{t}]$$






We can apply Tweedie's formula:
- with a variance of $\sigma^2 = (1-\bar{\alpha_t})$
- with the reparameterization  $X = \sqrt{\bar{\alpha_t}}X_{0}$ previously mentioned
$$ \mathbb{E}(X | X + N =x_t) = x_t + {\sigma^2} \nabla \log p_{\sigma^2}(X=x_t)$$



$$\widehat{x_0}(x_t)  = \frac{1}{\sqrt{\bar{\alpha_t}}} \left[ x_t + { (1-\bar{\alpha_t})} \nabla \log p_{(1-\bar{\alpha_t})}(X=x_t)\right]$$ 
If we look cautiously, this is not exactly the result we'd expect as **a blurred probability appears**!

with $ p_{(1-\bar{\alpha_t})}(X=x_t) = p_{(1-\bar{\alpha_t})}(X_0=\frac{x_t}{\sqrt{\bar{\alpha_t}}})$
Here the smoothed probability of $X_0$ appears which is equal to the probability $p(X_t=x_t)$ of the **non smoothed but noisy version** $X_t$ defined by the forward process  (adding Gaussian noise is equivalent to convolve the distribution by a gaussian blur).

Hence the final result

$$\widehat{x_0}(x_t)  = \frac{1}{\sqrt{\bar{\alpha_t}}} \left[ x_t + { (1-\bar{\alpha_t})} \nabla \log p(x_t)\right]$$ 

### Question 2

Forward process : $x_t = \sqrt{\bar\alpha_t} x_0 + \sqrt{1-\bar\alpha_t}\epsilon_t$
$$x_{0} = \frac{x_{t} - \sqrt{1-\bar\alpha_t} \epsilon_t}{\sqrt{\bar\alpha_t}}$$

If we have an estimation of the noise $\epsilon$ we should be able to recover the clean signal... 

$$\epsilon_t = \frac{x_t - \sqrt{ \bar\alpha_t}x_0}{\sqrt{1-\bar\alpha_t}}$$

### Question 3


- From question 1: $$ \widehat{x_0}(x_t) = \mathbb{E}[x_0|x_t] = \frac{1}{\sqrt{\bar\alpha_t}} \left( x_t + (1-\bar\alpha_t) \nabla \log p(x_t) \right) $$
- From question 2: $$x_{0} = \frac{x_{t} - \sqrt{1-\bar\alpha_t} \epsilon_t}{\sqrt{\bar\alpha_t}}$$


-----

-  $\widehat{x_0}(x_t) = \mathbb{E}[x_0|x_t] = \frac{1}{\sqrt{\bar\alpha_t}} \mathbb{E}[x_{t} - \sqrt{1-\bar\alpha_t} \epsilon_t | x_t] $
   -  by definition of $\widehat{x_0}(x_t)$
   -  by substitution of the result from question 2 for $x_0$ under the expectation symbol. 
-  $\sqrt{\bar\alpha_t}  \widehat{x_0}(x_t)  = x_{t}  - \sqrt{1-\bar\alpha_t} \mathbb{E}[ \epsilon_t | x_t]  $ 
   -  by linearity of the expectation
   -  and since $\mathbb{E}[X|X=x] = x$
-  $\sqrt{\bar\alpha_t}  \widehat{x_0}(x_t) = x_t + (1-\bar\alpha_t) \nabla \log p(x_t)$ *from question 1*
-  We can remove $x_t$ on both sides.
-  $(1-\bar\alpha_t) \nabla \log p(x_t) = -\sqrt{1-\bar\alpha_t} \mathbb{E}[ \epsilon_t | x_t]$
-  $\Leftrightarrow  \nabla \log p(x_t) = -\frac{1}{\sqrt{1-\bar\alpha_t}} \mathbb{E}[ \epsilon_t | x_t] $

$\mathbb{E}[ \epsilon_t | x_t] = \hat{\epsilon}(x_t)  \approx \hat{\epsilon}_{\theta}(x_t, t) $ is the expectation of the noise realization. This will be approximated by the neural network trained using MMSE. At that point, I think that the equal symbol does not stand and that we start making approximations.



__Result__
$$\nabla \log p(x_t) = - \frac{1}{\sqrt{1-\bar\alpha_t}} \hat\epsilon_\theta(x_t)$$

------

### Question 4

Update equation can be rewritten as:

$$x_{t-1} = \frac{1}{\sqrt{\alpha_t}} \left(x_{t} - \frac{\beta_t}{\sqrt{1 - \bar{\alpha_t}}}  \hat\epsilon \right) + \sigma_t z_t = \frac{1}{\sqrt{\alpha_t}} \left(x_{t} + \underbrace{\beta_t}_{1-\alpha_t} . \underbrace{\left[- \frac{1}{\sqrt{1 - \bar{\alpha_t}}}  \hat\epsilon\right]}_{\nabla \log p(x_t)} \right) + \sigma_t z_t$$

 
$\rightarrow x_{t-1} = \frac{1}{\sqrt{\alpha_t}} \left(x_{t} + (1 - \alpha_t) \nabla \log p(x_t) \right) + \sigma_t z_t$

Looks familiar... Looks like stochastic gradient ascent as we're trying to maximize the likelihood of$x_t$ to belong to the image distribution   (stochastic because of the $\sigma_t z_t$ noise term).


### Sampling
![](figures/alpha_schedule_small.png)

| Timesteps | 1000 | 100 | 10 | 3 | 2 |
|:---:| :--:|:--:|:--: | :--:| :--:|
|Generation time on Nvidia T500 | 5 min 30 | 32s | 2s | $< 1s$ | $< 1s$ |
| Generated image |![](figures/dpm_sample.png)|  ![](figures/dpm_sample_t100.png)|  ![](figures/dpm_sample_t10.png)|![](figures/dpm_sample_t3.png)|![](figures/dpm_sample_t2.png)|


It is important to take the steps to respect the diffusion framework conditions. If we reduce the amount of steps, we end up with a lack of details.

-----
# Diffusion models for inverse problems

![](figures/degradation.png)

Degradation: Large motion blur with AWGN

### Question 5
Let's check on a synthetic unitary test that we can compute the gradient of the "forward model" (blurring operator) with regard to the 
input.

See the code below for more details.

![](figures/blurt_kernel.png)
![](figures/deblur_data_term_gradient.png)



In [ ]:
# Apply the degradation model to an image and optimize a purely noisy prediction input to match the output by gradient descent
# This allows checking that we can compute the gradient g of f.
# Note that the gradient of a convolution is the transposed convolution.
# Here the blur being large, the convolution is performed as an iFFT(FFT(x) x FFT(k)) 
import utils.utils_agem as agem
import torch
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
kernel  = torch.zeros(1, 1, 9, 9)
diag_size  = 4
for idx in range(-(diag_size//2), diag_size//2+1+1):
    kernel[0, 0, kernel.shape[-2]//2+idx, kernel.shape[-1]//2+idx] = abs(idx)
kernel/=kernel.sum()
kernel.requires_grad = False
forward_model = lambda x: agem.fft_blur(x, kernel.to(device))
# inp = torch.rand(1, 1, 10, 10).to(device
inp = torch.zeros(1, 3, 32, 32).to(device)
inp[..., inp.shape[2]//4:3*inp.shape[2]//4, inp.shape[3]//4:3*inp.shape[3]//4] = 1.
out = forward_model(inp)
plt.figure(figsize=(10, 10/3))
plt.subplot(131)
plt.imshow(inp[0, 0].cpu().numpy(), cmap='gray')
plt.title('Groundtruth')
plt.axis('off')
plt.subplot(132)
plt.imshow(kernel[0, 0].cpu().numpy())
plt.title('Blur kernel')
plt.axis('off')
plt.subplot(133)
plt.imshow(out[0, 0].cpu().numpy(), cmap='gray')
plt.title('Degradation')
plt.axis('off')
plt.show()

pred = 0.2*torch.rand_like(inp)
plt.figure(figsize=(30, 3))
pred.requires_grad = True
f = torch.norm(forward_model(pred) - out)
for i in range(1001):
    if i%100 == 0:
        plt.subplot(1,11,i//100+1)
        plt.imshow(pred[0, 0].detach().cpu().numpy(), cmap='gray')
        plt.title(f'{i} step | loss {f.item():.3f}')
        plt.axis('off')
    f.backward()
    pred.data -= 0.1*pred.grad
    pred.grad.zero_()
    f = torch.norm(forward_model(pred) - out)
plt.suptitle('Optimization of the data term (blur) using the degradation model')
plt.show()

### Question 6
$\eta_t = \frac{\beta_t}{\sqrt{\alpha_t}}$